# ABOUT

this kernel is to preprocess DICOM images and 3D plot with ploty from the competition [OSIC Pulmonary Fibrosis Progression](https://www.kaggle.com/c/osic-pulmonary-fibrosis-progression).

DICON images are cropped, resized, segmentated, converted into Poly3DCollection and then saved into npy files for later use in deep learning.

I will keep working on it and improve preprocessings

Note that it is highly based on [OSIC |quick EDA + 3D plot with plotly](https://www.kaggle.com/sunpnwt12/osic-quick-eda-3d-plot-with-plotly)

# Import

In [3]:
# !conda install -c conda-forge gdcm -y # run this code for the first time
import os
# import gdcm
from collections import defaultdict
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import figure_factory as FF

import scipy.ndimage
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import random
import pydicom

# Read files

In [4]:
# load train and test image

DICOM_DIR = '../dataset/train'
# DICOM_DIR_TEST = '../data/test'

dicom_dict = defaultdict(list)
dicom_dict_test = defaultdict(list)

default_image_size = 512

for dirname in os.listdir(DICOM_DIR):
    path = os.path.join(DICOM_DIR, dirname)
    dicom_dict[dirname].append(path)
    
# for dirname in os.listdir(DICOM_DIR_TEST):
#     path = os.path.join(DICOM_DIR_TEST, dirname)
#     dicom_dict_test[dirname].append(path)


In [5]:
# ### load_scan:
# since there are a couple of dicom files that don't have 'ImagePositionPatient' attribute so instead,
# i will use 'InstanceNumber' attribute for those

# ### dicom_file:
# 1. take index number of patient which stored in dict_dicom earlier
# 2. this might be useful when you need to pick some random patient
# 3. It also takes specific patient Id in case you need.
# 4. Note that this function is going to read all file in taken path.

# ### get_pixels_hu
# 1. take dicom file which had called through dicom_file function
# 2. It stacks up all the load slices of certain patient
# 3. stacked slices will be calculated into Hounsfield Units



def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    length = len(slices)
    actualLength = sum([1 if hasattr(x, 'ImagePositionPatient') else 0 for x in slices])
    if length == actualLength:
        slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    else:
        slices.sort(key = lambda x: float(x.InstanceNumber))
        
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image < -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slbice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

# 3D Visualization

Useful links: 

* https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/
* https://medium.com/@hengloose/a-comprehensive-starter-guide-to-visualizing-and-analyzing-dicom-images-in-python-7a8430fcb7ed

In [6]:
# test = load_scan(dicom_dict['ID00014637202177757139317'][0])

# test_hu = get_pixels_hu(test)
# # print('Patient {}'.format(test[0].PatientName))
# # print('Slices : {}\nPixels : ({} x {})'.format(test_hu.shape[0], test_hu.shape[1], test_hu.shape[2]))

# # plt.figure(figsize=(12, 8))
# # ax = sns.distplot(test_hu.flatten(), bins=80, norm_hist=True)
# # # ax.set_title('Hounsfield Units of patient {}'.format(test_hu[0].PatientName), fontsize=25)
# # plt.show()

In [7]:
# ref) https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
# INTER_NEAREST - a nearest-neighbor interpolation
# INTER_LINEAR - a bilinear interpolation (used by default)
# INTER_AREA - resampling using pixel area relation. It may be a preferred method for image decimation, as it gives moire’-free results. But when the image is zoomed, it is similar to the INTER_NEAREST method.
# INTER_CUBIC - a bicubic interpolation over 4x4 pixel neighborhood
# INTER_LANCZOS4 - a Lanczos interpolation over 8x8 pixel neighborhood

import cv2
def resize(slices):
    new_slice = []
    for slice_number in range(len(slices)):
        new_slice.append(cv2.resize(slices[slice_number], dsize=(default_image_size, default_image_size), interpolation=cv2.INTER_AREA))
    return np.array(new_slice)

In [8]:
# # do crop here
def crop(slices):
    slice_height = len(slices[0])
    slice_width = len(slices[0][0])
    diff_height_half = int((slice_height - default_image_size) / 2)
    diff_width_half = int((slice_width - default_image_size) / 2)
    new_slice = []
    for slice_number in range(len(slices)):
        new_slice.append(slices[slice_number][diff_width_half: slice_width - diff_width_half, diff_height_half: slice_height - diff_height_half])
    return np.array(new_slice)

In [9]:
def resample(image, slice_tickness, x_pixel_spacing, y_pixel_spacing, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([slice_tickness, x_pixel_spacing, y_pixel_spacing], dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [10]:
def make_mesh(image, threshold):
    p = image.transpose(2, 1, 0)
    
    verts, faces, normals, values = measure.marching_cubes_lewiner(p, threshold)
    return verts, faces

def static_3d(image, threshold=-300):
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    verts, faces = make_mesh(image, threshold)
    x, y, z = zip(*verts)
    
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    
    ax.add_collection3d(mesh)
    ax.set_xlim(0, max(x))
    ax.set_ylim(0, max(y))
    ax.set_zlim(0, max(z))
    plt.show()
    
def interactive_3d(image, threshold=-300):
    verts, faces = make_mesh(image, threshold)
    x, y, z = zip(*verts)
    fig = FF.create_trisurf(x=x,
                            y=y,
                            z=z,
                            plot_edges=False,
                            simplices=faces)
    iplot(fig)

In [11]:
# x_size = test[0].Columns
# y_size = test[0].Rows
# slice_thinkness = test[0].SliceThickness
# x_pixel_spacing = test[0].PixelSpacing[0] * (x_size / default_image_size)
# y_pixel_spacing = test[0].PixelSpacing[1] * (y_size / default_image_size)
# # x_pixel_spacing = test[0].PixelSpacing[0]
# # y_pixel_spacing = test[0].PixelSpacing[1]
# print(test_hu.shape)
# print(x_pixel_spacing, y_pixel_spacing)

# # if ratio is not 1:1 then crop
# if (x_size != y_size):
#     test_hu = crop(test_hu)
#     # if cropped, reset pixel spacing with cropped image size because images with margin merely have space
#     # outter size of image so pixel spacing size shouldn't be affected by those space
#     x_pixel_spacing = test[0].PixelSpacing[0] * (test_hu.shape[2] / default_image_size)
#     y_pixel_spacing = test[0].PixelSpacing[1] * (test_hu.shape[1] / default_image_size)
    
# # if size is not 512 then resize
# if (x_size != default_image_size or y_size != default_image_size):
#     test_hu = resize(test_hu)

# # resample test
# resampled_test_hu, spacing = resample(test_hu, slice_thinkness, x_pixel_spacing, y_pixel_spacing)
# print(resampled_test_hu.shape)


In [12]:
# from scipy.ndimage import zoom
# z_med_size = 135
# y_med_size = 180
# x_med_size = 180
# shape = resampled_test_hu.shape
# print(z_med_size/shape[0], y_med_size/shape[1])
# resampled_test_hu = zoom(resampled_test_hu, (z_med_size/shape[0], y_med_size/shape[1], x_med_size/shape[2]))

The ratio of images that are not 1:1 are the ones with margins outside of images. crop those margins and resize to 512 resolution

As total resolution gets smaller, I muliplied pixel spacings by the ration too ( I guess as total number of pixel gets smaller, single pixel covers larger space thus bigger millimetres ) 

In [13]:
# resampled_test_hu.shape

In [14]:
# static_3d(resampled_test_hu)

In [15]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    z, y, x = labels.shape
    for slice_number in range(len(image)):
        #Fill the air around the person
        binary_image[slice_number][labels[slice_number, 0, 0] == labels[slice_number]] = 2
        binary_image[slice_number][labels[slice_number, 0, x - 1] == labels[slice_number]] = 2
        binary_image[slice_number][labels[slice_number, y - 1, 0] == labels[slice_number]] = 2
        binary_image[slice_number][labels[slice_number, y - 1, x - 1] == labels[slice_number]] = 2
        binary_image[slice_number][labels[slice_number, 0, int((x - 1) / 2)] == labels[slice_number]] = 2
        binary_image[slice_number][labels[slice_number, y - 1, int((x - 1) / 2)] == labels[slice_number]] = 2
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1
    return binary_image

In [16]:
# segmented_lungs = segment_lung_mask(resampled_test_hu, False)
# segmented_lungs_fill = segment_lung_mask(resampled_test_hu, True)

## Segmented lungs

In [17]:
# static_3d(segmented_lungs, 1.5)

## Segmented lungs filled

In [18]:
# static_3d(segmented_lungs_fill, 1.5)

## Difference between both

In [19]:
# static_3d(segmented_lungs_fill - segmented_lungs, -0.5)

In [20]:
# interactive_3d(segmented_lungs_fill - segmented_lungs, -0.5)

 # Save to npy


the codes below is to save segmented lung data into npy files.

In [21]:
# create directories
# os.removedirs("/kaggle/working/train")
# os.removedirs("/kaggle/working/test")
# os.makedirs('train')
# os.makedirs('test')

In [ ]:
# load all dicom
# import gdcm
from scipy.ndimage import zoom

PATH = '../dataset/output/train'
# PATH_TEST = '../dataset/output/test'

z_med_size = 135
y_med_size = 180
x_med_size = 180

# create train data npy
for k, v in dicom_dict.items():
    print(k ,v)
    if (k != 'ID00011637202177653955184' and k != 'ID00052637202186188008618'):
        
        scan = load_scan(v[0])
        scan_hu = get_pixels_hu(scan)

        x_size = scan[0].Columns
        y_size = scan[0].Rows
        slice_thinkness = scan[0].SliceThickness
        x_pixel_spacing = scan[0].PixelSpacing[0] * (x_size / default_image_size)
        y_pixel_spacing = scan[0].PixelSpacing[1] * (y_size / default_image_size)

        # # if ratio is not 1:1 then crop
        if (x_size != y_size):
            scan_hu = crop(scan_hu)
            # if cropped, reset pixel spacing with cropped image size because images with margin merely have space
            # outter side of image so pixel spacing size shouldn't be affected by those space
            x_pixel_spacing = scan[0].PixelSpacing[0] * (scan_hu.shape[2] / default_image_size)
            y_pixel_spacing = scan[0].PixelSpacing[1] * (scan_hu.shape[1] / default_image_size)

        # # if size is not 512 then resize
        if (x_size != default_image_size or y_size != default_image_size):
            scan_hu = resize(scan_hu)

        # # resample test
        resampled_scan_hu, spacing = resample(scan_hu, slice_thinkness, x_pixel_spacing, y_pixel_spacing)
        print('before resize', resampled_scan_hu.shape)

        z_ratio = z_med_size / resampled_scan_hu.shape[0]
        y_ratio = y_med_size / resampled_scan_hu.shape[1]
        x_ratio = x_med_size / resampled_scan_hu.shape[2]
        resampled_scan_hu = zoom(resampled_scan_hu, (z_ratio, y_ratio, x_ratio))
        print('after resize', resampled_scan_hu.shape)
        
        segmented_lungs = segment_lung_mask(resampled_scan_hu, False)
        segmented_lungs_fill = segment_lung_mask(resampled_scan_hu, True)
        np.save(f'{PATH}/{k}' , (segmented_lungs_fill - segmented_lungs) * -1)

ID00296637202279895784347 ['../dataset/train/ID00296637202279895784347']
before resize (31, 366, 366)
after resize (135, 180, 180)
ID00032637202181710233084 ['../dataset/train/ID00032637202181710233084']
before resize (256, 433, 433)
after resize (135, 180, 180)
ID00061637202188184085559 ['../dataset/train/ID00061637202188184085559']
before resize (632, 389, 389)
after resize (135, 180, 180)
ID00232637202260377586117 ['../dataset/train/ID00232637202260377586117']
before resize (335, 350, 350)
after resize (135, 180, 180)
ID00322637202284842245491 ['../dataset/train/ID00322637202284842245491']
before resize (265, 354, 354)
after resize (135, 180, 180)
ID00329637202285906759848 ['../dataset/train/ID00329637202285906759848']
before resize (260, 385, 385)
after resize (135, 180, 180)
ID00027637202179689871102 ['../dataset/train/ID00027637202179689871102']
before resize (716, 454, 454)
after resize (135, 180, 180)
ID00011637202177653955184 ['../dataset/train/ID00011637202177653955184']
ID00

before resize (251, 303, 303)
after resize (135, 180, 180)
ID00371637202296828615743 ['../dataset/train/ID00371637202296828615743']
before resize (364, 336, 336)
after resize (135, 180, 180)
ID00335637202286784464927 ['../dataset/train/ID00335637202286784464927']
before resize (30, 359, 359)
after resize (135, 180, 180)
ID00419637202311204720264 ['../dataset/train/ID00419637202311204720264']
before resize (35, 420, 420)
after resize (135, 180, 180)
ID00134637202223873059688 ['../dataset/train/ID00134637202223873059688']
before resize (451, 379, 379)
after resize (135, 180, 180)
ID00169637202238024117706 ['../dataset/train/ID00169637202238024117706']
before resize (288, 348, 348)
after resize (135, 180, 180)
ID00401637202305320178010 ['../dataset/train/ID00401637202305320178010']
before resize (30, 360, 360)
after resize (135, 180, 180)
ID00307637202282126172865 ['../dataset/train/ID00307637202282126172865']
before resize (29, 362, 362)
after resize (135, 180, 180)
ID0020763720225252638

ID00149637202232704462834 ['../dataset/train/ID00149637202232704462834']
before resize (306, 300, 300)
after resize (135, 180, 180)
ID00026637202179561894768 ['../dataset/train/ID00026637202179561894768']
before resize (239, 374, 374)
after resize (135, 180, 180)
ID00060637202187965290703 ['../dataset/train/ID00060637202187965290703']
before resize (275, 366, 366)
after resize (135, 180, 180)
ID00229637202260254240583 ['../dataset/train/ID00229637202260254240583']
before resize (17, 400, 400)
after resize (135, 180, 180)
ID00140637202231728595149 ['../dataset/train/ID00140637202231728595149']
before resize (813, 379, 379)
after resize (135, 180, 180)
ID00007637202177411956430 ['../dataset/train/ID00007637202177411956430']
before resize (38, 334, 334)
after resize (135, 180, 180)
ID00020637202178344345685 ['../dataset/train/ID00020637202178344345685']
before resize (246, 320, 320)
after resize (135, 180, 180)
ID00370637202296737666151 ['../dataset/train/ID00370637202296737666151']
befor

In [ ]:
# # load all dicom
# # import gdcm

# PATH = '../data/output/train'
# PATH_TEST = '../data/output/test'


# keys = []
# shapes = []

# # create train data npy
# for k, v in dicom_dict.items():
#     print(k ,v)
#     if (k != 'ID00011637202177653955184' and k != 'ID00052637202186188008618'):
        
#         scan = load_scan(v[0])
#         scan_hu = get_pixels_hu(scan)

#         x_size = scan[0].Columns
#         y_size = scan[0].Rows
#         slice_thinkness = scan[0].SliceThickness
#         x_pixel_spacing = scan[0].PixelSpacing[0] * (x_size / default_image_size)
#         y_pixel_spacing = scan[0].PixelSpacing[1] * (y_size / default_image_size)
# #         x_pixel_spacing = scan[0].PixelSpacing[0]
# #         y_pixel_spacing = scan[0].PixelSpacing[1]

#         # # if ratio is not 1:1 then crop
#         if (x_size != y_size):
#             scan_hu = crop(scan_hu)

#         # # if size is not 512 then resize
#         if (x_size != default_image_size or y_size != default_image_size):
#             scan_hu = resize(scan_hu)

#         # # resample test
#         resampled_test_hu, spacing = resample(scan_hu, slice_thinkness, x_pixel_spacing, y_pixel_spacing)
#         print(resampled_test_hu.shape)
# #         if (resampled_test_hu.shape[1] < minimum[1]):
# #             minimum = resampled_test_hu.shape
# #             min_key = k
# #         if(resampled_test_hu.shape[1] > maximum[1]):
# #             maximum = resampled_test_hu.shape
# #             max_key = k
            
# #         if (resampled_test_hu.shape[0] < z_minimum[0]):
# #             z_minimum = resampled_test_hu.shape
# #             z_min_key = k
# #         if(resampled_test_hu.shape[0] > z_maximum[0]):
# #             z_maximum = resampled_test_hu.shape
# #             z_max_key = k
#         keys.append(k)
#         shapes.append(resampled_test_hu.shape)


# # print('minimum', min_key, minimum)
# # print('maximim', max_key, maximum)
# # print('z_minimum', z_min_key, z_minimum)
# # print('z_maximim', z_max_key, z_maximum)
# #     segmented_lungs = segment_lung_mask(resample_scan_hu, False)
# #     segmented_lungs_fill = segment_lung_mask(resample_scan_hu, True)
# #     np.save(f'{PATH}/{k}' , segmented_lungs_fill - segmented_lungs)
    
# # create test data npy
# # for k, v in dicom_dict_test.items():
# #     print(k ,v)
# #     scan = load_scan(v[0])
# #     scan_hu = get_pixels_hu(scan)
# #     resample_scan_hu, spacing = resample(scan_hu, scan)
# #     segmented_lungs = segment_lung_mask(resample_scan_hu, False)
# #     segmented_lungs_fill = segment_lung_mask(resample_scan_hu, True)
# #     np.save(f'{PATH_TEST}/{k}' , segmented_lungs_fill - segmented_lungs)

In [ ]:
# x_median = np.median([x[1] for x in shapes])
# z_median = np.median([x[0] for x in shapes])